In [4]:
import pandas as pd
import psycopg2

import warnings
warnings.filterwarnings("ignore")

In [180]:
energy_raw = pd.read_csv("./data/raw/Electric_Consumption_And_Cost__2010_-_Feb_2023__20240417.csv")
energy_raw.head(3)

,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Other charges
0,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,15396.82,GOV/NYC/068,Yes,128800.0,7387.97,216.0,2808.0,5200.85
1,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,14556.34,GOV/NYC/068,Yes,115200.0,6607.87,224.0,2912.0,5036.47
2,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,13904.98,GOV/NYC/068,Yes,103200.0,5919.55,216.0,2808.0,5177.43


In [181]:
energy_raw.columns

Index(['Development Name', 'Borough', 'Account Name', 'Location', 'Meter AMR',
       'Meter Scope', 'TDS #', 'EDP', 'RC Code', 'Funding Source', 'AMP #',
       'Vendor Name', 'UMIS BILL ID', 'Revenue Month', 'Service Start Date',
       'Service End Date', '# days', 'Meter Number', 'Estimated',
       'Current Charges', 'Rate Class', 'Bill Analyzed', 'Consumption (KWH)',
       'KWH Charges', 'Consumption (KW)', 'KW Charges', 'Other charges'],
      dtype='object')

In [182]:
energy_raw.shape

(447849, 27)

## Data Cleaning Tasks 

#### Task 1: Identify and Remove Irrelevant Features

In [183]:
energy_selected_column =energy_raw[["Borough", "Account Name","Meter Number","Funding Source","Current Charges",\
                                     'Consumption (KWH)','KWH Charges', 'Consumption (KW)', 'KW Charges',\
                                     "Revenue Month","Service Start Date", "Service End Date"]]
energy_selected_column.head()

,Borough,Account Name,Meter Number,Funding Source,Current Charges,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Revenue Month,Service Start Date,Service End Date
0,BRONX,ADAMS,7223256,FEDERAL,15396.82,128800.0,7387.97,216.0,2808.0,2010-01,12/24/09,1/26/10
1,BRONX,ADAMS,7223256,FEDERAL,14556.34,115200.0,6607.87,224.0,2912.0,2010-02,1/26/10,2/25/10
2,BRONX,ADAMS,7223256,FEDERAL,13904.98,103200.0,5919.55,216.0,2808.0,2010-03,2/25/10,3/26/10
3,BRONX,ADAMS,7223256,FEDERAL,14764.04,105600.0,6057.22,208.0,2704.0,2010-04,3/26/10,4/26/10
4,BRONX,ADAMS,7223256,FEDERAL,13729.54,97600.0,5598.34,216.0,2808.0,2010-05,4/26/10,5/24/10


### Task 1.1: Remove columns

In [184]:
energy_selected_column.rename(columns= 
                                {"Borough":"borough", 
                                "Account Name" :"account_name",
                                "Meter Number":"meter_number",
                                "Funding Source":"funding_source",
                                "Current Charges":"current_charges",
                                "Revenue Month":"revenue_month",
                                "Service Start Date":"service_start_date",
                                "Service End Date":"service_end_date"},inplace = True)

energy_selected_column.head(2)
#TODO: fix the rename the name - KWH and KW

,borough,account_name,meter_number,funding_source,current_charges,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,revenue_month,service_start_date,service_end_date
0,BRONX,ADAMS,7223256,FEDERAL,15396.82,128800.0,7387.97,216.0,2808.0,2010-01,12/24/09,1/26/10
1,BRONX,ADAMS,7223256,FEDERAL,14556.34,115200.0,6607.87,224.0,2912.0,2010-02,1/26/10,2/25/10


### Task 2: Handle Missing Data

In [185]:
energy_handle_missing_data = energy_selected_column.copy()
energy_handle_missing_data.isnull().sum().sum()

16

In [186]:
energy_handle_missing_data.isnull().sum()

borough               0
account_name          0
meter_number          0
funding_source        0
current_charges       0
Consumption (KWH)     0
KWH Charges           0
Consumption (KW)      0
KW Charges            0
revenue_month         0
service_start_date    8
service_end_date      8
dtype: int64

In [176]:
energy_handle_missing_data[energy_handle_missing_data.service_start_date .isnull()]

,borough,account_name,meter_number,funding_source,current_charges,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,revenue_month,service_start_date,service_end_date
294254,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.00,0.0,0.0,0.0,0.0,2020-04,NaN,NaN
297117,BROOKLYN,RED HOOK EAST,9956008,FEDERAL,15.96,0.0,0.0,0.0,0.0,2020-02,NaN,NaN
297120,BROOKLYN,RED HOOK EAST,9956007,FEDERAL,20.22,0.0,0.0,0.0,0.0,2020-02,NaN,NaN
337554,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.00,0.0,0.0,0.0,0.0,2020-04,NaN,NaN
337555,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.04,0.0,0.0,0.0,0.0,2020-06,NaN,NaN
337556,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.00,0.0,0.0,0.0,0.0,2020-05,NaN,NaN
347588,BROOKLYN,RED HOOK EAST,9956008,FEDERAL,15.96,0.0,0.0,0.0,0.0,2020-02,NaN,NaN
347592,BROOKLYN,RED HOOK EAST,9956007,FEDERAL,20.22,0.0,0.0,0.0,0.0,2020-02,NaN,NaN


In [177]:
energy_handle_missing_data.shape

(447849, 12)

In [157]:
energy_handle_missing_data.fillna({
    'estimated':'unknown', 'tds_number':'unknown',
    'service_start_date':'unknown', 'service_end_date':'unknown'
}, inplace=True)

In [158]:
energy_handle_missing_data.isna().sum()

borough               0
account_name          0
meter_number          0
funding_source        0
current_charges       0
tds_number            0
revenue_month         0
estimated             0
service_start_date    0
service_end_date      0
dtype: int64

In [156]:
energy_handle_missing_data[energy_handle_missing_data['estimated'] == 'unknown']

,borough,account_name,meter_number,funding_source,current_charges,tds_number,revenue_month,estimated,service_start_date,service_end_date
413,MANHATTAN,HARLEM RIVER,7195851,FEDERAL,0.0,3.0,2010-08,unknown,7/23/10,8/23/10
441,MANHATTAN,HARLEM RIVER,7195852,FEDERAL,0.0,3.0,2010-08,unknown,7/23/10,8/23/10
469,MANHATTAN,HARLEM RIVER,7195889,FEDERAL,0.0,3.0,2010-08,unknown,7/23/10,8/23/10
497,MANHATTAN,HARLEM RIVER,7195891,FEDERAL,0.0,3.0,2010-08,unknown,7/23/10,8/23/10
525,MANHATTAN,HARLEM RIVER,7195894,FEDERAL,0.0,3.0,2010-08,unknown,7/23/10,8/23/10
...,...,...,...,...,...,...,...,...,...,...
294254,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.0,337.0,2020-04,unknown,NaN,NaN
294255,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.0,337.0,2020-02,unknown,1/24/20,2/25/20
337554,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.0,337.0,2020-04,unknown,NaN,NaN
337557,MANHATTAN,LOWER EAST SIDE II,5503302,FEDERAL,0.0,337.0,2020-02,unknown,1/24/20,2/25/20


In [150]:
mean_value = energy_handle_missing_data['tds_number'].mean()
energy_handle_missing_data_2 = energy_handle_missing_data.copy()
energy_handle_missing_data_2['tds_number '].fillna(mean_value, inplace=True)

KeyError: 'tds_number '

In [118]:
energy_missing_data.info()

<class 'pandas.core.series.Series'>
Index: 10 entries, borough to service_end_date
Series name: None
Non-Null Count  Dtype
--------------  -----
10 non-null     int64
dtypes: int64(1)
memory usage: 460.0+ bytes


### Task 1.2: fix the data type

In [9]:
energy_selected_column['service_start_date'] = pd.to_datetime(energy_selected_column['service_start_date'])
energy_selected_column['service_end_date'] = pd.to_datetime(energy_selected_column['service_end_date'])


energy_selected_column['revenue_month'] = energy_selected_column['revenue_month'].astype(str) + '-01'
energy_selected_column['revenue_month'] = pd.to_datetime(energy_selected_column['revenue_month'])
#energy_selected_column['revenue_month']  = energy_selected_column['revenue_month'].dt.strftime('%Y-%m-%d')


energy_selected_column['current_charges'] = energy_selected_column ['current_charges'].astype(float)

energy_selected_column.dtypes

borough                       object
account_name                  object
meter_number                  object
funding_source                object
current_charges              float64
tds_number                   float64
revenue_month         datetime64[ns]
estimated                     object
service_start_date    datetime64[ns]
service_end_date      datetime64[ns]
dtype: object

In [10]:
energy_selected_column.head()

,borough,account_name,meter_number,funding_source,current_charges,tds_number,revenue_month,estimated,service_start_date,service_end_date
0,BRONX,ADAMS,7223256,FEDERAL,15396.82,118.0,2010-01-01,N,2009-12-24,2010-01-26
1,BRONX,ADAMS,7223256,FEDERAL,14556.34,118.0,2010-02-01,N,2010-01-26,2010-02-25
2,BRONX,ADAMS,7223256,FEDERAL,13904.98,118.0,2010-03-01,N,2010-02-25,2010-03-26
3,BRONX,ADAMS,7223256,FEDERAL,14764.04,118.0,2010-04-01,N,2010-03-26,2010-04-26
4,BRONX,ADAMS,7223256,FEDERAL,13729.54,118.0,2010-05-01,N,2010-04-26,2010-05-24


In [11]:
energy_40 = energy_selected_column.head(40)

In [12]:
# # Connection parameters
# conn_params = {
#     'dbname': 'nyc_energy',
#     'user': 'postgres',
#     'password': 'postgres',         
#     'host': 'localhost',
#     'port': '5432'
# }

# # Connect to the PostgreSQL database
# conn = psycopg2.connect(**conn_params)
# cur = conn.cursor()

In [13]:
# # Create the 'nyc_energy database
# cur.execute("CREATE DATABASE  nyc_energy")

# # Commit the changes and close the connection to the default database
# conn.commit()
# cur.close()
# conn.close()

In [14]:
energy_40.dtypes

borough                       object
account_name                  object
meter_number                  object
funding_source                object
current_charges              float64
tds_number                   float64
revenue_month         datetime64[ns]
estimated                     object
service_start_date    datetime64[ns]
service_end_date      datetime64[ns]
dtype: object

In [15]:
energy_40.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   borough             40 non-null     object        
 1   account_name        40 non-null     object        
 2   meter_number        40 non-null     object        
 3   funding_source      40 non-null     object        
 4   current_charges     40 non-null     float64       
 5   tds_number          40 non-null     float64       
 6   revenue_month       40 non-null     datetime64[ns]
 7   estimated           40 non-null     object        
 8   service_start_date  40 non-null     datetime64[ns]
 9   service_end_date    40 non-null     datetime64[ns]
dtypes: datetime64[ns](3), float64(2), object(5)
memory usage: 3.3+ KB


## load dataframe into postgres


In [14]:
import pandas as pd
import psycopg2
from psycopg2 import extras


 # Connect to PostgreSQL
conn = psycopg2.connect(
    dbname='nyc_energy',
    user='postgres',
    password='postgres',
    host='localhost',
    port='5432'
)

# Create a cursor for database operations
cur = conn.cursor()

# Define the PostgreSQL table structure (if needed)
create_table_query = """
CREATE TABLE IF NOT EXISTS raw_energy_data (
    id SERIAL PRIMARY KEY,
    borough VARCHAR(100),
    account_name VARCHAR(100),
    funding_source VARCHAR(100), 
    meter_number INT,
    revenue_month DATE,
    service_end_date DATE,
    service_start_date DATE,
    tds_number FLOAT,
    current_charges FLOAT
)
"""
cur.execute(create_table_query)

# Define the insertion query for bulk insertion
insert_query = """
INSERT INTO raw_energy_data (borough, account_name, funding_source, meter_number, revenue_month, service_end_date, service_start_date, tds_number, current_charges) VALUES %s
"""

# Define the insertion query for bulk insertion
values = [tuple(row) for row in energy_40[['borough', 'account_name', 'funding_source', 'meter_number', 'revenue_month', 'service_end_date', 'service_start_date', 'tds_number', 'current_charges']].values]

# Execute the insertion query with corrected values
extras.execute_values(cur, insert_query, values)

# Commit changes
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

print("Data inserted successfully.")

Data inserted successfully.


In [15]:
energy_selected_column[['borough', 'meter_number']].values

array([['BRONX', '7223256'],
       ['BRONX', '7223256'],
       ['BRONX', '7223256'],
       ...,
       ['BROOKLYN', '2786583'],
       ['BROOKLYN', '2786584'],
       ['BROOKLYN', '2786586']], dtype=object)

In [16]:
[tuple(row) for row in energy_selected_column[['borough', 'meter_number']].head(10).values]

[('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256')]